# INFO 3401 – Class 26: Pointplots and heatmaps

[Brian C. Keegan, Ph.D.](http://brianckeegan.com/)  
[Assistant Professor, Department of Information Science](https://www.colorado.edu/cmci/people/information-science/brian-c-keegan)  
University of Colorado Boulder  

Copyright and distributed under an [MIT License](https://opensource.org/licenses/MIT).

## Learning Objectives
* Working with spatial point data in GeoPandas
* Visualizing spatial data as pointplots, sometimes in combination with choropleths
* Visualizing spatial point data as heatmaps and kdeplots

## Background

This module will explore how to acquire, analyze, and visualize spatial data. I have adapted this content from the excellent course [Auotmating GIS processes](https://automating-gis-processes.github.io/site/index.html) course by [Vuokko Heikinheimo](https://researchportal.helsinki.fi/en/persons/vuokko-vilhelmiina-heikinheimo) and [Henrikki Tenkanen](https://www.ucl.ac.uk/geospatial-analytics/people/henrikki-tenkanen).

## Load libraries

Load our usual libraries.

In [ ]:
# Data processing libraries
import pandas as pd
import numpy as np

pd.options.display.max_columns = 100

# Visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

# Geospatial libraries
import geopandas as gpd
from pyproj import CRS
import contextily, geoplot

import matplotlib.colors as colors

## Load data

We're going to use the National Highway Traffic Safety Administration's databases on fatal highway accidents. [Data going back to 1976 is available](https://www.nhtsa.gov/research-data/fatality-analysis-reporting-system-fars), but we'll focus on data since 2012. There's also a detailed [user manual](https://www.nhtsa.gov/filebrowser/download/176821) describing these columns if you're interested: the data we're using correspdong to the "ACCIDENT" data files. The Appendix section at the end of the notebook details how this data was generated.

In [ ]:
fars_df = pd.read_csv('fars_2012-2018.csv')

print(fars_df.shape)

fars_df.head()

Load the US shapefile too.

In [ ]:
states_gdf = gpd.read_file('us_states/tl_2019_us_state.shp')

states_gdf.head(1)

Finally load the Colorado counties and convert the "county" name to title-case.

In [ ]:
co_counties_gdf = gpd.read_file('co_counties/co_counties.shp')

co_counties_gdf['county'] = co_counties_gdf['county'].str.title()

co_counties_gdf.head(1)

## Feature engineering

We need to convert some of the columns in this data into more useful data types.

First, take the "YEAR", "MONTH", "DAY", "HOUR", and "MINUTE" columns and use `pd.to_datetime` to combine them into a "Timestamp" column.

Second, there are columns for "LATITUDE" (y-axis) and "LONGITUD" (x-axis). Use GeoPandas's [`points_from_xy`](https://geopandas.org/reference/geopandas.points_from_xy.html) function (also this tutorial [Creating a GeoDataFrame from a DataFrame with coordinate](https://geopandas.org/gallery/create_geopandas_from_pandas.html)). Convert these to a shapely Point geometry and convert the DataFrame into a GeoDataFrame by specifying both the geometry and the CRS.

Third, there are more than 200,000 fatal car accidents in the six years of data across the entire U.S. Rename the "State Name" and "County" names to just "State" and "County" for simplicity. Then filter down to only accidents in Colorado.

## Exploratory analysis

Check the distribution of incidents by "County" and "YEAR".

Use a groupby on the "Timestamp" column with a `Grouper` to plot the number of fatalities ("FATALS") per day.

Try out some other methods like a rolling average to help clarify the visualization.

Make a pivot table with "DAY_WEEK" as columns, "HOUR" as index, "FATALS" as values, and 'sum' as an aggfunc. Visualize as a heatmap. Sunday is 1 and Saturday is 7.

## Point plot

Visualize the individual incidents.

Visualize using the Colorado state boundaries.

Visualize using a basemap. You may need to convert to another CRS like [EPSG:3857](https://epsg.io/3857) used by Google, OSM, etc.

## Make a density plot

Use geoplot's [`kdeplot`](https://residentmario.github.io/geoplot/plot_references/plot_reference.html#kdeplot) ([tutorial](https://geopandas.org/gallery/plotting_with_geoplot.html)) to make a density plot of these points. Use geoplot's `webmap` to include a basemap.

## Convert to a choropleth

We have done joins above and before in previous classes, but spatial data presents a new way to join data as well. A [spatial join](https://geopandas.org/mergingdata.html#spatial-joins) combines and merges data based on their respective geometries: is a Point within a Polygon for example? This is valuable if we want to take spatial data like accidents and aggregate to count the number of incidents per county. 

A spatial join allows us to merge the `co_counties_gdf` GeoDataFrame and the `co_fars_gdf` GeoDataFrame. Because the `co_fars_gdf` already has a column for "County" this feels a bit redundant, but because spatial joins are so powerful, I still want to cover how to do them and use the existing labels for verification.

Inspect the `co_counties_gdf`.

Inspect the `co_fars_gdf`.

What does a spatial join do under the hood?

Do the `sjoin` with `co_fars_gdf` on the left, `co_counties_gdf` on the right, using a "left" join, and an "intersects" operation.

Compare.

Where are these errors? They all appear to be on county boundary lines/roads.

Group total fatalities by county and merge back into `co_counties_gdf`.

Visualize county-level data as a choropleth.

## Appendix

Read in the files and concatenate together.

In [ ]:
fars_accident_l = []

for y in range(2012,2019):
    _df = pd.read_csv('fars_{0}_accident.csv'.format(y))
    fars_accident_l.append(_df)
    
fars_accident_df = pd.concat(fars_accident_l,ignore_index=True)
fars_accident_df.head()

Get the GSA [Geographic Locator Codes](https://www.gsa.gov/reference/geographic-locator-codes-glcs-overview) for the states and counties used in the FARS data. These are at City-level, so make a DataFrame of just State and County codes.

In [ ]:
glc_df = pd.read_excel('https://www.gsa.gov/cdnstatic/FRPP_GLC_-_United_StatesSep292020.xlsx')

# Make title-cased
glc_df['State Name'] = glc_df['State Name'].str.title()
glc_df['City Name'] = glc_df['City Name'].str.title()
glc_df['County Name'] = glc_df['County Name'].str.title()

# Groupby state and county
glc_state_county = glc_df.groupby(['State Code','County Code']).agg({'State Name':'first','County Name':'first'}).reset_index()


Merge in state and county codes with the FARS data.

In [ ]:
labeled_fars_df = pd.merge(left = fars_accident_df,
         right = glc_state_county,
         how = 'left',
         left_on = ['STATE','COUNTY'],
         right_on = ['State Code','County Code']
        )

labeled_fars_df.drop(columns=['timestamp','geometry','State Code','County Code'],inplace=True)

labeled_fars_df.to_csv('fars_2012-2018.csv',index=False)

labeled_fars_df.head()